In [1]:
import qiskit
# from qiskit_ibm_provider import IBMProvider
# from qiskit import IBMQ
# import qiskit.tools.jupyter
from qiskit.tools.parallel import parallel_map
import json
from styles.style import qspain

from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import COBYLA, SPSA, POWELL
from qiskit.primitives import Estimator

from qiskit_nature.second_q.circuit.library import UCC, PUCCD, HartreeFock
from qiskit_nature.second_q.mappers import QubitMapper, JordanWignerMapper
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

import pennylane as qml
from pennylane import numpy as np


%matplotlib inline
#IBMQ.load_account()

#%qiskit_backend_overview

/tmp/ipykernel_11450/537049398.py:9: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms.minimum_eigensolvers import VQE


In [2]:
'''Pennylane_code_cell'''

# Construimos la molécula de Hidrógeno H2
symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 1.400])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)

# Obtenemos el estado de referencia de Hartree-Fock
electrons = 2
ref_state = qml.qchem.hf_state(electrons, qubits)

In [3]:
'''Pennylane_code_cell'''

# En pennylane tenemos que definir el circuito sobre el que trabajará el ansatz
dev = qml.device('default.qubit', wires=qubits)

# Definimos el ansatz k-UpCCGSD
# weights son los parámetros t^q_p, t^{q_\alpha q_\beta}_{p_\alpha p_\beta}
# wires son los qubits (o en general los sistemas) sobre los que se define el ansatz.
# delta_sz especifica las reglas de selección para la proyección del espín en dirección z 
#     en los orbitales de las excitaciones simples.

k = 3

@qml.qnode(dev)
def ansatz(weights):
    qml.kUpCCGSD(weights, wires=[0, 1, 2, 3],
                    k=k, delta_sz=0, init_state=ref_state)
    return qml.expval(H)

shape = qml.kUpCCGSD.shape(k=k,
                    n_wires=qubits, delta_sz=0)

In [4]:
'''Pennylane_code_cell'''

# Escogemos un optimizador
opt = qml.GradientDescentOptimizer(stepsize=0.4)
# opt = qml.GradientDescentOptimizer(stepsize=0.04)
# opt = qml.QNGOptimizer(stepsize=0.01)
# opt = qml.RMSPropOptimizer(stepsize=0.01)
# opt = qml.AdamOptimizer(stepsize=0.01)
# opt = qml.AdagradOptimizer(stepsize=0.01)
# opt = qml.SPSAOptimizer(maxiter=250) 

In [5]:
'''Pennylane_code_cell'''

# Inicializamos los primeros parámetros
weights = np.random.random(size=shape)
energy = [ansatz(weights)]

# E iniciamos el proceso iterativo
angle = [weights]
max_iterations = 250
conv_tol = 1e-08 # Escogemos una tolerancia en la que queremos que se detenga el algoritmo
for n in range(max_iterations):
    weights, prev_energy = opt.step_and_cost(ansatz, weights)
    energy.append(ansatz(weights))
    angle.append(weights)
    conv = np.abs(energy[-1] - prev_energy)
    # if n % 4 == 0:
        # print(f"Iteración: {n},  Energía = {energy[-1]:.8f} Ha")
    if conv <= conv_tol:
        break

print("\n" f"Energía del estado fundamental (obtenida) = {energy[-1]:.8f} Ha")
print("\n" f"Valores óptimos del circuito = {angle[-1]}")


Energía del estado fundamental (obtenida) = -1.13727592 Ha

Valores óptimos del circuito = [[0.64602866 0.34079656 0.42082063 0.16922154 0.62086335 0.3463154 ]
 [0.2275569  0.04592067 0.12976618 0.68158533 0.29590652 0.35328037]
 [0.39530577 0.64276052 0.72391752 0.18231534 0.23546012 0.22749385]]
